In [ ]:
# Import necessary libraries
import pandas as pd
import os
import platform
device_name = platform.node()

previous_folder = os.getcwd()
print ("This is the working folder: " + previous_folder)

if device_name == 'mmd-MS-7D98': 
    #This passage is done because everytime I log into the remote server the default folder is:
    #/mmd/home and I need to localize the correct folder to load the settings 
    os.chdir("/media/mmd/Samsung_T5/GitHub/UMD")

if previous_folder != os.getcwd(): # This is now the right working folder
    print("The current working folder has been changed, now the working folder is: " + os.getcwd())

In [ ]:
import sys
sys.path.append("..")
sys.path.append("my_library")
from my_library.config import *
from my_library.metrics.cmp_metrics import *
sys.path.append("..")

Clear example of spaghetti code
I had to rush to obtain the statistics

In [ ]:
#eval = "ALLeval/eval/test-int"
#eval = "ALLeval/eval_15folds/test-int"
eval = "ALLeval/eval_max50slides/test-int"
#eval = "ALLeval/eval_1_single_fold/test-int"
#setup ="1 Fold"
#setup ="15 Fold"
setup = "5 Folds-Max 50 Tiles"
#setup = "5 Folds"

#ile_name = "ABMIL_results_test_set_all models_15_fold.xlsx"
file_name = "ABMIL_results_test_set_all models_Max50tiles_fold.xlsx"
#file_name = "ABMIL_results_test_single_model.xlsx"
#file_name = "ABMIL_results_test_set_all models.xlsx"

In [ ]:
base_folder = os.path.join(working_directory,eval)
subfolder = [name for name in os.listdir(base_folder) if os.path.isdir(os.path.join(base_folder, name))]

results = []
intermediate_values = []
df_test_list = []

for m in subfolder:
    df = []
    print(m)
    root_dir = os.path.join(base_folder, m)

    %run -i './metriche/from_parquet.ipynb'

    #print(parent_directory)
    #compute_metrics(parent_directory, n_classes, parent_directory)
    results.append(df)
    df_test_list.append(df_test)
    intermediate_values.append((m, y_true, y_pred, outcome_labels))


In [ ]:
intermediate_values[3]

In [ ]:
for i in intermediate_values:
    model, y_true, y_pred, outcome_labels = i

    print(outcome_labels)
    
    labels = [idx for idx, val in enumerate(sorted(set(outcome_labels)))]
    conf_matrix = confusion_matrix(y_true, y_pred, labels=labels)
    conf_matrix = np.array(conf_matrix)
    plot_confusion_matrix_complex(conf_matrix, outcome_labels,  model, setup, working_directory = working_directory, cmap = "YlGnBu")


    if model == "pdl1-3":
        outcome_labels = np.array(list(outcome_labels))
        outcome_labels[0], outcome_labels[1] = outcome_labels[1], outcome_labels[0]

        conf_matrix[[0, 1]] = conf_matrix[[1, 0]]
        plot_confusion_matrix_complex(conf_matrix, outcome_labels,  model, setup, working_directory = working_directory, cmap = "YlGnBu")
    

In [ ]:
"""conf_matrix = confusion_matrix(y_true, y_pred, labels=[0,1,2])
conf_matrix = np.array(conf_matrix)
plot_confusion_matrix_complex(conf_matrix, outcome_labels,  m, "15 Folds", "YlGnBu")

outcome_labels = np.array(list(outcome_labels))
outcome_labels[0], outcome_labels[1] = outcome_labels[1], outcome_labels[0]

conf_matrix[[0, 1]] = conf_matrix[[1, 0]]
plot_confusion_matrix_complex(conf_matrix, outcome_labels,  m, "15 Folds", "YlGnBu")"""

In [ ]:
subfolder

In [ ]:
from re import sub

df_list = []

for i, m in zip(results, subfolder):
    transpose_df = i.T
    transpose_df = transpose_df.rename(columns={0: m})
    #display(transpose_df)
    transpose_df = transpose_df.reset_index()
    transpose_df.rename(columns={transpose_df.columns[0]: 'MyModel'}, inplace=True)

    df_list.append(transpose_df)




In [ ]:
df_list[3]

In [ ]:
result_df_1 = pd.concat([df_list[0]] , axis=1)
result_df_2 = pd.concat([df_list[1]] , axis=1)
result_df_3 = pd.concat([df_list[2]] , axis=1)
result_df_4 = pd.concat([df_list[3]] , axis=1)

In [ ]:
with pd.ExcelWriter(file_name) as writer:
    result_df_1.to_excel(writer, sheet_name="Best Response 3", index=False)
    result_df_2.to_excel(writer, sheet_name="Adeno vs Squamous", index=False)
    result_df_3.to_excel(writer, sheet_name="PDL1 3 classes", index=False)
    result_df_4.to_excel(writer, sheet_name="PDL1 2 classes", index=False)

In [ ]:
subfolder

In [ ]:
df_test_list[0]

In [ ]:
df_test_list[2]

In [ ]:
csv_df = pd.read_csv(f"{working_directory}/projects/I3lung-sqadqc-project/annotations.csv")

In [ ]:
file_path = working_directory+"/data_config.json"

import json

# Open and read the JSON file
with open(file_path, "r") as file:
    data = json.load(file)  # Load JSON data into a Python dictionary or list

print(data)
data.keys()

In [ ]:
import json

In [ ]:
labels = list(data["labels"].values())

In [ ]:
labels

In [ ]:
from anyio import value
from pandas import value_counts

id_list_list = []

for i,l in zip(df_test_list, labels):
    df_joined = pd.merge(i, csv_df, on ='slide', how = 'left')
    selected_df = df_joined[["slide", l]]
    value_counts = selected_df[l].value_counts()
    df_counts = value_counts.to_frame(name="Count")
    df_counts["Percentage (%)"] = (df_counts["Count"] / df_counts["Count"].sum()) * 100

    #display(selected_df)
    print(selected_df[l].value_counts())
    display(df_counts)

        # Convertire la colonna "ID" in una lista
    id_list = df_joined["slide"].tolist()
    id_list = [i+".pt" for i in id_list]

    # Mostrare la lista risultante
    id_list_list.append(id_list)

In [ ]:
# Aggiungere ".pt" a tutte le stringhe nella colonna "ID"
#df_joined["slide"] = df_joined["slide"] + ".pt"

In [ ]:
import torch

In [ ]:
shape_list_list = []
folder_path = "/media/mmd/Extreme SSD/work/bags/uni/full-qc/10x reinhard_fast"
for list_dir in id_list_list:
    shape_list = []
    for filename in os.listdir(folder_path):

        if filename in list_dir:
            file_path = os.path.join(folder_path, filename)
            
            # Load the PyTorch object
            data = torch.load(file_path)
            
            # If each .pt file stores a *single* tensor, print its shape
            if hasattr(data, 'shape'):
                #print(f"{filename} => shape: {tuple(data.shape)}")
                shape_list.append(data.shape[0])
            #else:
                # If data isn't a single tensor, just show what was loaded
                #print(f"{filename} => not a single tensor (type: {type(data)})")
    print(len(shape_list))
    shape_list_list.append(shape_list)


In [ ]:
values = shape_list_list[0]
# Create the histogram
plt.figure(figsize=(10, 8))

n, bins, patches = plt.hist(values, bins=10)  # You can adjust 'bins' to change how many bars appear

#plt.xticks(bins)
import matplotlib as mpl
mpl.rcParams['font.size'] = 16

# Annotate each bar with its count
for i, patch in enumerate(patches):
    # Find the center of each bin
    bin_center = (bins[i] + bins[i+1]) / 2
    
    # Label just above the top of each bar
    plt.text(bin_center, n[i], int(n[i]), 
             ha='center', va='bottom', )#fontsize=12)
plt.xticks(fontsize=24)
plt.yticks(fontsize=24)
# Add axis labels and a title
plt.xlabel("$N_i$", fontsize=24)
plt.ylabel("Frequency", fontsize = 24)
#plt.title("Histogram of Values")

# Display the plot
plt.savefig("histogram_tiles.png", transparent=True)
plt.show()